# Market basket analysis using apriori algorithm

About this Dataset
• The Online Retail II data set, which includes the sales data of the UK-based online sales store, was used.
• Sales data between 01/12/2010 - 09/12/2011 are included in the data set.
• The product catalog of this company includes souvenirs.
• There is also information that most of its customers are wholesalers.

Variables:
• InvoiceNo: Invoice Number -> If this code starts with C, it means that the operation has been canceled.
• StockCode: Product Code -> Unique number for each product
• Description: Product name
• Quantity: Number of products -> how many of the products on the invoices were sold.
• InvoiceDate
• UnitePrice
• CustomerID: Unique customer number
• Country

### Import required libraries

In [81]:
import pandas as pd

# For Association Rules Learning & Apriori 
from mlxtend.frequent_patterns import apriori, association_rules

### Import dataset in pandas dataframe

In [3]:
df = pd.read_excel('C:/Users/apoor/Desktop/Apoorva/Learnings/Apriori algo/online_retail_II/online_retail_II.xlsx')

### To check non-null values and datatypes of columns

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


There are 541,910 non null rows except Description and Customer ID which contains null values

### To check the no of null values in each column

In [6]:
df.isna().sum() 

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

Description contains 0.2% of null rows and Customer Id contains 25% of null rows 

### Drop null values in the dataframe

In [7]:
df.dropna(inplace=True)

In [9]:
df.shape

(406830, 8)

Finally dataframe contains 75% of the initial data(541,910 rows) aftre removing duplicates

### Check statistics of quantative columns

In [87]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,357405.0,11.794236,25.567641,1.000,2.00,5.00,12.00,298.50
Price,357405.0,2.827035,3.099559,0.001,1.25,1.69,3.75,37.06
Customer ID,357405.0,15303.358674,1710.679299,12346.000,13982.00,15159.00,16806.00,18287.00


### Remove cancelled transactions

Invoice id contains "C" as first character for all cancelled transactions as described in data description

In [88]:
df_valid = pd.DataFrame({"Invoice":[row for row in df["Invoice"].values if "C"  not in str(row)]})
df_valid = df_valid.drop_duplicates("Invoice")

Replace dataframe with only valid transactions (not cancelled)

In [90]:
df = df.merge(df_valid, on = "Invoice")

### Outlier detection

Defining first quartile at 1% and third quartile at 99% and defining upper and lower limit

In [91]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

Replacing outliers with upper and lower limit

In [93]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit 

Checking datatypes of dataframe 

In [95]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

To store int and float datatypes columns in list (except ID)

In [97]:
columns_num = [col for col in df.columns if df[col].dtypes in ["int64","float64"] and "ID" not in col]
print(columns_num)

['Quantity', 'Price']


Call outlier functions defined above to replace outliers with threshold values

In [98]:
for col in columns_num:
    replace_with_thresholds(df, col)

### To check number of products same as of number of stockcodes

In [102]:
df.Description.nunique()

3419

In [103]:
df.StockCode.nunique()

3419

There are 3,877 unique descriptions and 3,665 unique stockcodes in dataframe. The unique count of these 2 variables should be same, because each stock code represents an item

To store the items with one description and multiple stockcode in different dataframe

In [105]:
df_multiple = df[["Description","StockCode"]].drop_duplicates()
df_multiple = df_multiple.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()
df_multiple.sort_values("StockCode", ascending=False).head()

,Description,StockCode
0,4 PURPLE FLOCK DINNER CANDLES,1
2271,PINK/WHITE RIBBED MELAMINE JUG,1
2273,PIZZA PLATE IN BOX,1
2274,PLACE SETTING WHITE HEART,1
2275,PLACE SETTING WHITE STAR,1


In [107]:
df_multiple.rename(columns={'StockCode':'StockCode_Count'},inplace=True)
df_multiple = df_multiple[df_multiple["StockCode_Count"]>1]

Delete items with more than one stockcode

In [43]:
df = df[~df["Description"].isin(df_multiple["Description"])]

Check unique count of descriptions and stockcodes

In [44]:
print(df.StockCode.nunique())
print(df.Description.nunique())

3630
3858


To store the items with one stockcode and multiple descriptions in different dataframe

In [109]:
df_multiple_d = df[["Description","StockCode"]].drop_duplicates()
df_multiple_d = df_multiple_d.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_multiple_d.rename(columns={'Description':'Description_Count'},inplace=True)

In [113]:
df_multiple_d = df_product.sort_values("Description_Count", ascending=False)
df_multiple_d = df_multiple_d[df_multiple_d["Description_Count"] > 1] 
df_multiple_d.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2129,23396,3
1881,23126,3
1963,23209,3


Delete items with more than one description

In [115]:
df = df[~df["StockCode"].isin(df_multiple_d["StockCode"])]

Check unique count of descriptions and stockcodes

In [49]:
print(df.StockCode.nunique())
print(df.Description.nunique())

3420
3420


The "post" in the stockcode indicates the postage cost in dataset, delete it as it is not a valid item

In [50]:
df = df[~df["StockCode"].str.contains("POST", na=False)]

Lets filter the dataframe for Germany only to minimize the size of the data

In [117]:
df_germany = df[df["Country"] == "Germany"]
df_germany.shape

(7843, 8)

### Create a function to prepare invoice-matrix for applying apriroi algo

In [118]:
def invoice_item_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [119]:
invoice_df = invoice_item_df(df_germany, id=True)
invoice_df.head()

StockCode,10002,10125,11001,15034,15036,15039,16008,16011,16014,16016,...,90160D,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Define a function to find the product name corresponding to the stockcode

In [121]:
def display_name(dataframe, stockcode):
    product_name = dataframe[dataframe["StockCode"] == stockcode]["Description"].unique()[0]
    return stockcode, product_name

In [122]:
display_name(df_germany, 10002)

(10002, 'INFLATABLE POLITICAL GLOBE ')

Get the support values for every possible configuration of items (thereshold of support is 0.05 (5%))

In [123]:
frequent_items = apriori(gr_inv_pro_df, min_support=0.05, use_colnames=True)

In [128]:
frequent_items.head()

,support,itemsets
0,0.103371,(20712)
1,0.130337,(20719)
2,0.071910,(20724)
3,0.080899,(20726)
4,0.067416,(20750)


Apply association rule

In [129]:
rules = association_rules(frequent_items, metric="support", min_threshold=0.05)
rules.sort_values("support", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(22328),(22326),0.161798,0.251685,0.134831,0.833333,3.311012,0.094109,4.489888
5,(22326),(22328),0.251685,0.161798,0.134831,0.535714,3.311012,0.094109,1.805359
6,(22554),(22326),0.141573,0.251685,0.076404,0.539683,2.144274,0.040773,1.625649
7,(22326),(22554),0.251685,0.141573,0.076404,0.303571,2.144274,0.040773,1.232613
10,(22629),(22326),0.105618,0.251685,0.071910,0.680851,2.705167,0.045328,2.344719


In [130]:
sorted_rules = rules.sort_values("lift", ascending=False)
sorted_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(20724),(20719),0.071910,0.130337,0.060674,0.843750,6.473599,0.051302,5.565843
1,(20719),(20724),0.130337,0.071910,0.060674,0.465517,6.473599,0.051302,1.736426
16,(22554),(22556),0.141573,0.119101,0.069663,0.492063,4.131476,0.052801,1.734270
17,(22556),(22554),0.119101,0.141573,0.069663,0.584906,4.131476,0.052801,2.068029
14,(22554),(22551),0.141573,0.110112,0.062921,0.444444,4.036281,0.047332,1.601798


To check the item name with its stockcode

In [137]:
product_id = 22554
check_id(df, product_id)

(22554, 'PLASTERS IN TIN WOODLAND ANIMALS')

To store the recommendations for one item

In [138]:
product_id = 22554
recommendation_list = []
for idx, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
            recommendation_list = list( dict.fromkeys(recommendation_list) )

In [139]:
list_top5 = recommendation_list[0:5]
for elem in list_top5:
    print(check_id(df_germany,elem))

(22556, 'PLASTERS IN TIN CIRCUS PARADE ')
(22551, 'PLASTERS IN TIN SPACEBOY')
(22326, 'ROUND SNACK BOXES SET OF4 WOODLAND ')


So, customers whi are purchasing item - 22554, the store can recommend them to purchase these 3 items- 22556, 22551, 22326